# Carga da tabela fact_horas de maneira FULL


> [!CAUTION]  
> Todos os dados serão sobrescritos

## Leitura do arquivo e carregado em 'staging_horas'  

In [ ]:
from pyspark.sql.types import *

# Define o schema do dataframe
csv_schema = StructType([ \
    StructField("Matricula", IntegerType(), True), \
    StructField("Data", DateType(), True), \
    StructField("HorasNormais", DoubleType(), True), \
    StructField("HorasDisponiveis", DoubleType(), True)
  ])

# Lê o arquivo
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ";") \
    .option("dateFormat", "dd/MM/yyyy") \
    .schema(csv_schema) \
    .load("Files/fact_horas.csv")

# Salva nas tabelas delta
df.write.mode('overwrite').saveAsTable("staging_horas")


## Cria a tabela e faz a transformação da 'fact_horas'  
> Busca a 'matricula_sk' na tabela 'dim_funcionarios' para correto relacionamento.  

In [ ]:
%%sql

-- Apaga a tabela se existir
DROP TABLE IF EXISTS fact_horas;

-- Cria a tabela a partir incluindo a 'MatriculaSk' da tabela 'dim_funcionarios'
CREATE TABLE fact_horas AS
SELECT 
    h.Matricula, 
    h.Data, 
    h.HorasNormais, 
    h.HorasDisponiveis,
    f.MatriculaSk
FROM 
    staging_horas h

LEFT JOIN dim_funcionarios f
    ON h.Matricula = f.Matricula
    AND h.Data BETWEEN f.DataVigenciaInicial AND f.DataVigenciaFinal;

SELECT * FROM fact_horas;

-- Apaga a tabela temporária
DROP TABLE IF EXISTS staging_horas;
